# 导入库及定义共享函数

In [1]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# 定义网络
#net trained by convnet-mnist
def convnet_mnist():
    net = input_data(shape=[None,28,28,1], name='input')
    net = conv_2d(net, 32,3, activation='relu', regularizer='L2')
    net = max_pool_2d(net,2)
    net = local_response_normalization(net)
    net = conv_2d(net,64,3, activation='relu', regularizer='L2')
    net = max_pool_2d(net,2)
    net = local_response_normalization(net)
    net = fully_connected(net, 128, activation='tanh')
    net = dropout(net, 0.8)
    net = fully_connected(net, 256, activation='tanh',name='feature')
    
    return net

In [3]:
# 定义网络
#net trained by cifar10-convnet-mnist
# Convolutional network building
def cifar10_convnet_mnist():
    network = input_data(shape=[None, 28, 28, 1])
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = dropout(network, 0.75)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='relu')
        
    #network = dropout(network, 0.5)
    #network = fully_connected(network, 6, activation='softmax',restore=False)
    #network = regression(network, optimizer='adam',
    #                     loss='categorical_crossentropy',
    #                    learning_rate=0.001)

    return network

In [10]:
# 把抽取的特征写入到arff格式的文件
# arffname: arff file's name
# features: 抽取出的特征
# label：样本标签（0或1）
# filemodel：文件读写模式，a,a+,w,r等等
def writeOneClassFeaturesToArffFile(arffame, features, label, filemodel):
    num_samples = len(features)
    num_features = len(features[0])
    arff_file = open(arffname,filemodel)
    
    if filemodel == 'w':

        arff_file.write('@relation relationship\n')

        for i in range(1,num_features+1):
            line = '@attribute ' + 'Att' + str(i) + ' numeric\n'
            arff_file.write(line)

        arff_file.write('@attribute class {0,1}\n')
        arff_file.write('@data\n\n')

    for i in range(num_samples):
        line = []
        for f in features[i]:
            line.append(str(f))
        
        line.append(str(label)) 
        arff_file.write(",".join(line))
        arff_file.write('\n')
    arff_file.close()    

In [5]:
# 从图像读写数据
import os
import numpy as np
from PIL import Image
def loadOneClassImageArray(filepath,num_feature, label):
    files = os.listdir(filepath)
    N = len(files)
    X = np.ndarray((N,num_feature),dtype=np.float32)
    
    i = 0
    for file in files:
        k = file.index('.')
        key = file[:k]
        fn = os.path.join(filepath,file)
        img = Image.open(fn,"r")
        m = np.array(img)
        m = m.reshape((1,num_feature))
        X[i] = m
        
        i = i + 1
    return X

# 从预训练的网络抽取6个活性的抗菌肽CA特征

In [ ]:
from prepareDataset import load_data
X,Y = load_data('e:/repoes/ampnet/data/img_60/', 'e:/repoes/ampnet/data/benchmark_60_Targets.json')
X = X.reshape((-1,28,28,1))

In [ ]:
# net trained by convnet-mnist
net = convnet_mnist()
model = tflearn.DNN(net, tensorboard_verbose=0)
model.load('e:/repoes/ampnet/model/convnet_mnist', weights_only=True)

In [ ]:
# 抽取特征，写入文件
features=model.predict(X)
arff_file = open('amp_convnet_mnist_features.arff','w')
arff_file.write('@relation relationship\n')
for i in range(1,257):
    line = '@attribute ' + 'Att' + str(i) + ' numeric\n'
    arff_file.write(line)
arff_file.write('@attribute class {0,1}\n')
arff_file.write('@data\n\n')

for i in range(len(X)):
    line = []
    for f in features[i]:
        line.append(str(f))
    for y in Y[i]:
        line.append(str(int(y))) 
    arff_file.write(",".join(line))
    arff_file.write('\n')
arff_file.close()    

In [ ]:
#net trained by cifar10-convnet-mnist
# Real-time data preprocessing
# Convolutional network building
network = cifar10_convnet_mnist()
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.load('e:/repoes/ampnet/model/cifar10_cnn_mnist', weights_only=True)

In [ ]:
features=model.predict(X)
arff_file = open('amp_cifa10_cnn_mnist_features.arff','w')
arff_file.write('@relation relationship\n')
for i in range(1,513):
    line = '@attribute ' + 'Att' + str(i) + ' numeric\n'
    arff_file.write(line)
arff_file.write('@attribute class {0,1}\n')
arff_file.write('@data\n\n')

for i in range(len(X)):
    line = []
    for f in features[i]:
        line.append(str(f))
    for y in Y[i]:
        line.append(str(int(y))) 
    arff_file.write(",".join(line))
    arff_file.write('\n')
arff_file.close()    

In [ ]:
Y_pred = model.predict(X)

# 从预训练的网络抽取正样本抗菌肽（AMPs）A和负样本非抗菌肽（notAMPs）的特征

In [6]:
network = cifar10_convnet_mnist()
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.load('e:/repoes/ampnet/model/cifar10_cnn_mnist', weights_only=True)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
INFO:tensorflow:Restoring parameters from e:/repoes/ampnet/model/cifar10_cnn_mnist


In [11]:
# 抽取特征，写入文件
#ampX = loadOneClassImageArray('e:/repoes/ampnet/data/img/AMPs_50',784,1)
#ampX = ampX.reshape((-1,28,28,1))
#features1 = model.predict(ampX)
arffname = 'e:/repoes/ampnet/amp_and_notamp.arff'
writeOneClassFeaturesToArffFile(arffname, features1, 1, 'w' )

notampX = loadOneClassImageArray('e:/repoes/ampnet/data/img/notAMPs_50',784,1)
notampX = ampX.reshape((-1,28,28,1))
features2 = model.predict(notampX)
writeOneClassFeaturesToArffFile(arffname, features2, 0, 'a' )

NameError: name 'features2' is not defined